<a href="https://colab.research.google.com/github/theindianwriter/CS6910-assignment_2/blob/main/DL_assignment_2_Part_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Librabries imported

In [1]:
import numpy as np #for linear algebra operations
import os
import shutil
import matplotlib.pyplot as plt
from PIL import Image #for preprocessing the images
import random

In [2]:
# PyTorch libraries
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import gc

In [ ]:
!pip install wandb -qqq
import wandb
wandb.login()

     |████████████████████████████████| 2.1MB 19.9MB/s 
     |████████████████████████████████| 163kB 57.3MB/s 
     |████████████████████████████████| 102kB 14.1MB/s 
     |████████████████████████████████| 133kB 57.2MB/s 
     |████████████████████████████████| 71kB 10.9MB/s 


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
#enabling gpu 
device = "cpu"
if (torch.cuda.is_available()):
    device = "cuda"

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip -uq "/content/drive/My Drive/nature_12K.zip" -d "/content/drive/My Drive/"

In [28]:
training_folder_path = "/content/drive/MyDrive/inaturalist_12K/train"
test_folder_path = "/content/drive/MyDrive/inaturalist_12K/test"
#validation_folder_path = "/content/drive/MyDrive/inaturalist_12K/validation"

In [6]:
#list down all the classes present in the dataset
classes = sorted([folder_name  for folder_name in os.listdir(training_folder_path) if not folder_name.startswith('.')])
print(classes)

['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']


In [7]:
# function to resize image
def resize_image(src_image, size=(128,128), bg_color="white"): 
    #creating a thumbnail of the image of the given size preserving the aspect ratio
    src_image.thumbnail(size,Image.ANTIALIAS)
    #creating a background image 
    new_image = Image.new("RGB", size, bg_color)
    #pasting the src image into it 
    new_image.paste(src_image, (int((size[0] - src_image.size[0]) / 2), int((size[1] - src_image.size[1]) / 2)))
    return new_image

In [25]:
new_training_folder_path = "/content/drive/MyDrive/Natural_Dataset256/train"
new_test_folder_path = '/content/drive/MyDrive/Natural_Dataset256/test'
#new_validation_folder_path = './Natural_Dataset256/validation'

In [9]:
def Resize_all(old_path,new_path,size=(256,256)):

    image_size = size

    # if os.path.exists(new_path):
    #     shutil.rmtree(new_path)
    print("the new path is ",new_path)
    for root,folders,_ in os.walk(old_path):
        for folder in folders:
            print("resizing the images and saving for the folder ",folder)
            new_folder = os.path.join(new_path,folder)

            if not os.path.exists(new_folder):
                os.makedirs(new_folder)

            # if os.path.exists(os.path.join(root,folder)):

        
            file_names = os.listdir(os.path.join(root,folder))

            for file_name in file_names:
                if file_name.startswith('.'):
                    continue
                
                file_path = os.path.join(root,folder,file_name)
                image = Image.open(file_path)
                resized_image = resize_image(image,image_size,"white")
                save_as = os.path.join(new_folder,file_name)
                resized_image.save(save_as)

    print("resizing and saving done")


In [13]:
Resize_all(training_folder_path,new_training_folder_path)
#Resize_all(validation_folder_path,new_validation_folder_path)

the new path is  /content/drive/MyDrive/Natural_Dataset256/train
resizing the images and saving for the folder  Reptilia
resizing the images and saving for the folder  Mammalia
resizing the images and saving for the folder  Arachnida
resizing the images and saving for the folder  Plantae
resizing the images and saving for the folder  Aves
resizing the images and saving for the folder  Amphibia
resizing the images and saving for the folder  Insecta
resizing the images and saving for the folder  Animalia
resizing the images and saving for the folder  Mollusca
resizing the images and saving for the folder  Fungi
resizing and saving done


In [29]:
Resize_all(test_folder_path,new_test_folder_path)

the new path is  /content/drive/MyDrive/Natural_Dataset256/test
resizing the images and saving for the folder  Amphibia
resizing the images and saving for the folder  Aves
resizing the images and saving for the folder  Plantae
resizing the images and saving for the folder  Insecta
resizing the images and saving for the folder  Reptilia
resizing the images and saving for the folder  Mammalia
resizing the images and saving for the folder  Mollusca
resizing the images and saving for the folder  Fungi
resizing the images and saving for the folder  Animalia
resizing the images and saving for the folder  Arachnida
resizing and saving done


In [31]:
mean = [0.5,0.5,0.5]
std = [0.5,0.5,0.5]

def load_dataset(dataset_path,data_augmentation = False,batch_size = 50,is_test = False):
    if data_augmentation:
        transformation = transforms.Compose([
            transforms.RandomResizedCrop(256),
            transforms.RandomHorizontalFlip(0.5),
            transforms.RandomVerticalFlip(0.3),
            transforms.ToTensor(),
        # Normalize the pixel values (in R, G, and B channels)
            transforms.Normalize(mean=mean, std=std)
        ])
    else:
        transformation = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=mean,std = std)                               
        ])

    full_dataset = torchvision.datasets.ImageFolder(
        root=dataset_path,
        transform=transformation
    )
    if is_test:
        test_loader = torch.utils.data.DataLoader(
            full_dataset,
            batch_size=batch_size,
            num_workers=0,
            shuffle=False
        ) 
        return test_loader

    train_size = int(0.9 * len(full_dataset))
    val_size = len(full_dataset) - train_size

    train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        num_workers=0,
        shuffle=False
    )

    val_loader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=batch_size,
        num_workers=0,
        shuffle=False
    )
    return train_loader,val_loader

In [15]:
print(device)

cuda


In [15]:
class Network(nn.Module):

    def __init__(self,num_of_classes,filters,filter_size,activation_fn,stride_len = 1,batch_norm = True,padding_len = 1,max_pool_kernel_size = 2,max_pool_stride_len = 2,dropdout_ratio = 0.0,dense_neuron_size = 20000,img_size = (256,256)):
        super(Network,self).__init__()

        self.activation_fn = activation_fn
        self.batch_norm = batch_norm

        Out_dim_H = img_size[0]
        Out_dim_W = img_size[1]

        for F in filter_size:
            Out_dim_H =  int(((Out_dim_H - F + 2*padding_len)/stride_len)) + 1
            Out_dim_W =  int(((Out_dim_W - F + 2*padding_len)/stride_len)) + 1
            Out_dim_H =  int(((Out_dim_H - max_pool_kernel_size)/max_pool_stride_len))+ 1
            Out_dim_W =  int(((Out_dim_W - max_pool_kernel_size)/max_pool_stride_len)) + 1


        self.conv1 = nn.Conv2d(in_channels = 3,out_channels = filters[0],kernel_size = filter_size[0],stride = stride_len,padding = padding_len)
        self.conv2 = nn.Conv2d(in_channels = filters[0],out_channels = filters[1],kernel_size = filter_size[1],stride = stride_len,padding = padding_len)
        self.conv3 = nn.Conv2d(in_channels = filters[1],out_channels = filters[2],kernel_size = filter_size[2],stride = stride_len,padding = padding_len)
        self.conv4 = nn.Conv2d(in_channels = filters[2],out_channels = filters[3],kernel_size = filter_size[3],stride = stride_len,padding = padding_len)
        self.conv5 = nn.Conv2d(in_channels = filters[3],out_channels = filters[4],kernel_size = filter_size[4],stride = stride_len,padding = padding_len)

        self.pool = nn.MaxPool2d(kernel_size=max_pool_kernel_size,stride = max_pool_stride_len)

        self.drop = nn.Dropout2d(p=dropdout_ratio)
        
        if self.batch_norm:
            self.batch_norm1 = nn.BatchNorm2d(num_features=filters[0], eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            self.batch_norm2 = nn.BatchNorm2d(num_features=filters[1], eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            self.batch_norm3 = nn.BatchNorm2d(num_features=filters[2], eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            self.batch_norm4 = nn.BatchNorm2d(num_features=filters[3], eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            self.batch_norm5 = nn.BatchNorm2d(num_features=filters[4], eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        

        self.post_conv_output_len = Out_dim_H*Out_dim_W*filters[4]
        #fully connected layer
        self.fc1 = nn.Linear(in_features = Out_dim_H*Out_dim_W*filters[4],out_features = dense_neuron_size)
        self.fc2 = nn.Linear(in_features = dense_neuron_size,out_features = num_of_classes)



    def perform_activation(self,fn,x):
        if fn == 'relu':
            return F.relu(x)
        elif fn == 'elu':
            return F.elu(x)
        elif fn == 'leaky_relu':
            return F.leaky_relu(x)
        elif fn == 'tanh':
            return torch.tanh(x)
        elif fn == 'sigmoid':
            return F.sigmoid(x)

    def forward(self,x):
        if self.batch_norm:
            x = self.batch_norm1(self.conv1(x))
        else:
            x = self.conv1(x)

        x = self.pool(self.perform_activation(self.activation_fn[0],x))

        if self.batch_norm:
            x = self.batch_norm2(self.conv2(x))
        else:
            x = self.conv2(x)
    
        x = self.pool(self.perform_activation(self.activation_fn[1],x))

        if self.batch_norm:
            x = self.batch_norm3(self.conv3(x))
        else:
            x = self.conv3(x)


        x = self.pool(self.perform_activation(self.activation_fn[2],x))

        if self.batch_norm:
            x = self.batch_norm4(self.conv4(x))
        else:
            x = self.conv4(x)

        x = self.pool(self.perform_activation(self.activation_fn[3],x))
    
        if self.batch_norm:
            x = self.batch_norm5(self.conv5(x))
        else:
            x = self.conv5(x)

        x = self.pool(self.perform_activation(self.activation_fn[4],x))

        x = F.dropout(self.drop(x), training=self.training)

        x = x.view(-1, self.post_conv_output_len)

        x = F.relu(self.fc1(x))

        x = F.dropout(self.drop(x), training=self.training)

        x = self.fc2(x)
            
        return torch.log_softmax(x, dim=1)

In [16]:
def train(model,train_loader,optimizer,epoch,device):

    model.train()
    training_loss = 0
    batch_id = 0
    print("----------------------IN EPOCH {}--------------------------------".format(epoch))
    for data,target in train_loader:
        batch_id += 1
        data,target = data.to(device),target.to(device)

        optimizer.zero_grad()
        output = model(data)

        loss = loss_criteria(output,target)
        training_loss += loss.item()

        loss.backward()
        optimizer.step()

    print("avg training loss is {:.6f}".format(training_loss/batch_id))
    return training_loss


In [17]:
def test(model,test_loader,device):
    model.eval()
    test_loss = 0
    correct = 0
    batch_id = 0
    with torch.no_grad():
        for data,target in test_loader:
            batch_id += 1
            data,target = data.to(device),target.to(device)
            output = model(data)
            loss = loss_criteria(output,target)
            test_loss += loss.item()
            _,predicted = torch.max(output.data,1)
            correct += torch.sum(target==predicted).item()
    accuracy = (100*correct/ len(test_loader.dataset))
    print("avg val loss is {:.6f} and accuracy is {:.2f}%".format(test_loss/batch_id,accuracy))
    return test_loss,accuracy

    


In [19]:
model = Network(num_of_classes=10,filters = [32,64,128,256,512],filter_size = [5,5,5,5,5],activation_fn=['tanh','tanh','tanh','tanh','tanh'],dropdout_ratio = 0.5,dense_neuron_size = 256,batch_norm = True).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

loss_criteria = nn.CrossEntropyLoss()
train_loader,val_loader = load_dataset(new_training_folder_path,False,200)
epochs = 20
print('Training on', device)
for epoch in range(1, epochs + 1):
        train_loss = train(model, train_loader, optimizer,epoch,device)
        test_loss = test(model, val_loader,device)
        torch.cuda.empty_cache()
        

Training on cuda
----------------------IN EPOCH 1--------------------------------
avg training loss is 2.255514
avg val loss is 2.092842 and accuracy is 27.40%
----------------------IN EPOCH 2--------------------------------
avg training loss is 2.157679
avg val loss is 2.034590 and accuracy is 28.30%
----------------------IN EPOCH 3--------------------------------
avg training loss is 2.126044
avg val loss is 2.000279 and accuracy is 30.70%
----------------------IN EPOCH 4--------------------------------
avg training loss is 2.098106
avg val loss is 1.988044 and accuracy is 30.80%
----------------------IN EPOCH 5--------------------------------
avg training loss is 2.074774
avg val loss is 1.967815 and accuracy is 31.30%
----------------------IN EPOCH 6--------------------------------
avg training loss is 2.055943
avg val loss is 1.972563 and accuracy is 29.70%
----------------------IN EPOCH 7--------------------------------
avg training loss is 2.039542
avg val loss is 1.939554 and a

In [23]:
torch.save(model,"/content/drive/MyDrive/Natural_Dataset256/Best_model/1")

In [ ]:
sweep_config = {
    'method': 'random', #grid, random
    'metric': {
      'name': 'accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'lr': {
            'values': [0.0001, 0.0002,0.0004]
        },
        'activation_fn': {
            'values': ['relu', 'tanh','leaky_relu']
        },
        'num_of_filters': {
            'values' : [32,64,128,256]
        },
        'filter_size' : {
            'values' : [3,5,7]
        },
        'filter_organization':{
            'values': [2,1]
        },
        'batch_size':{
            'values': [100,200]
        },
        'dropdout_ratio': {
            'values': [0.2,0.3,0.4,0.5]
        },
        'dense_neuron_size':{
            'values' : [256,128,64]
        },
        'data_augmentation':{
            'values': [True,False]
        },
        'epochs':{
            'values' : [10,5,15]
        },
        'batch_norm':{
            'values' : [True,False]
        },


    }
}

In [ ]:
def experiment():
    defaults = dict(
        num_of_filter = 32,
        filter_size = 5,
        filter_organization = 2,
        activation_fn='tanh',
        dropout_ratio = 0.3,
        dense_neuron_size = 64,
        lr = 0.0001,
        batch_size = 8,
        epochs = 10,
        data_augmentation = True,
        batch_norm = True
    )

    wandb.init(project="cs6910-assignment2-part_a",config = defaults)
    config = wandb.config

    filters = []
    filter_size = []
    activation_fn = []  
    rate = 1
    for i in range(5):
        activation_fn.append(config.activation_fn)
        filter_size.append(config.filter_size)
        filters.append(config.num_of_filter*rate)
        rate *= config.filter_organization

    model = Network(num_of_classes=10,filters = filters,filter_size = filter_size,activation_fn=activation_fn,dropdout_ratio = config.dropout_ratio,dense_neuron_size = config.dense_neuron_size,batch_norm = config.batch_norm).to(device)
    optimizer = optim.Adam(model.parameters(), lr=config.lr)

    loss_criteria = nn.CrossEntropyLoss()
    train_loader,val_loader = load_dataset(new_training_folder_path,config.data_augmentation,config.batch_size)
    epochs = config.epochs
    print('Training on', device)
    for epoch in range(1, epochs + 1):
        gc.collect()
        torch.cuda.empty_cache()
        train_loss = train(model, train_loader, optimizer,epoch,device)
        val_loss,accuracy = test(model, val_loader,device)
        wandb.log({"epoch": epoch,"accuracy" : accuracy,"training loss":train_loss,"validation loss": val_loss })
        
    del model

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity="theindianwriter", project="cs6910-assignment2-part_a")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: fm8gabmx
Sweep URL: https://wandb.ai/theindianwriter/cs6910-assignment2-part_a/sweeps/fm8gabmx


In [ ]:
wandb.agent(sweep_id, experiment)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: kmdlu97u with config:
wandb: 	activation_fn: relu
wandb: 	batch_norm: True
wandb: 	batch_size: 100
wandb: 	data_augmentation: True
wandb: 	dense_neuron_size: 64
wandb: 	dropdout_ratio: 0.3
wandb: 	epochs: 10
wandb: 	filter_organization: 1
wandb: 	filter_size: 5
wandb: 	lr: 0.0001
wandb: 	num_of_filters: 256
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Training on cuda
----------------------IN EPOCH 1--------------------------------
avg training loss is 2.352233
avg val loss is 2.280871 and accuracy is 19.70%
----------------------IN EPOCH 2--------------------------------
avg training loss is 2.288860
avg val loss is 2.255837 and accuracy is 21.50%
----------------------IN EPOCH 3--------------------------------
avg training loss is 2.265626
avg val loss is 2.224513 and accuracy is 24.50%
----------------------IN EPOCH 4--------------------------------
avg training loss is 2.254328
avg val loss is 2.204476 and accuracy is 25.40%
----------------------IN EPOCH 5--------------------------------
avg training loss is 2.239879
avg val loss is 2.181257 and accuracy is 24.40%
----------------------IN EPOCH 6--------------------------------
avg training loss is 2.225736
avg val loss is 2.162044 and accuracy is 26.90%
----------------------IN EPOCH 7--------------------------------
avg training loss is 2.208331
avg val loss is 2.148858 and a

epoch,10
accuracy,28.5
training loss,197.1502
validation loss,21.12257
_runtime,849
_timestamp,1618307762
_step,9


epoch,▁▂▃▃▄▅▆▆▇█
accuracy,▁▂▄▅▄▆█▆▇▇
training loss,█▅▄▄▃▃▂▂▁▁
validation loss,█▇▆▅▄▃▃▂▁▁
_runtime,▁▂▃▃▄▅▆▆▇█
_timestamp,▁▂▃▃▄▅▆▆▇█
_step,▁▂▃▃▄▅▆▆▇█


wandb: Agent Starting Run: l10mf64o with config:
wandb: 	activation_fn: tanh
wandb: 	batch_norm: True
wandb: 	batch_size: 100
wandb: 	data_augmentation: True
wandb: 	dense_neuron_size: 256
wandb: 	dropdout_ratio: 0.2
wandb: 	epochs: 10
wandb: 	filter_organization: 0.5
wandb: 	filter_size: 3
wandb: 	lr: 0.0001
wandb: 	num_of_filters: 256
wandb: WARNING Ignored wandb.init() arg project when running a sweep


...

epoch,15
accuracy,39.8
training loss,86.87661
validation loss,9.34846
_runtime,1126
_timestamp,1618323119
_step,14


epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
accuracy,▁▅▄▄▅▆▇▇▇▇▇▇█▇█
training loss,█▇▆▅▅▄▄▃▃▃▂▂▂▁▁
validation loss,█▆▆▅▅▄▄▄▃▃▂▃▁▃▂
_runtime,▁▁▂▂▃▃▄▅▅▆▆▇▇██
_timestamp,▁▁▂▂▃▃▄▅▅▆▆▇▇██
_step,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 72vsfb6x with config:
wandb: 	activation_fn: tanh
wandb: 	batch_norm: False
wandb: 	batch_size: 100
wandb: 	data_augmentation: True
wandb: 	dense_neuron_size: 64
wandb: 	dropdout_ratio: 0.4
wandb: 	epochs: 15
wandb: 	filter_organization: 0.5
wandb: 	filter_size: 5
wandb: 	lr: 0.0001
wandb: 	num_of_filters: 64
wandb: WARNING Ignored wandb.init() arg project when running a sweep


wandb: ERROR Run 72vsfb6x errored: TypeError('new() received an invalid combination of arguments - got (float, int, int, int), but expected one of:\n * (*, torch.device device)\n * (torch.Storage storage)\n * (Tensor other)\n * (tuple of ints size, *, torch.device device)\n * (object data, *, torch.device device)\n')
wandb: Agent Starting Run: dkse6hmd with config:
wandb: 	activation_fn: relu
wandb: 	batch_norm: True
wandb: 	batch_size: 100
wandb: 	data_augmentation: True
wandb: 	dense_neuron_size: 128
wandb: 	dropdout_ratio: 0.4
wandb: 	epochs: 5
wandb: 	filter_organization: 2
wandb: 	filter_size: 7
wandb: 	lr: 0.0001
wandb: 	num_of_filters: 64
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Training on cuda
----------------------IN EPOCH 1--------------------------------


In [34]:
test_loader = load_dataset(new_test_folder_path,batch_size=200,is_test=True)

test_loss,accuracy = test(model=model,test_loader=test_loader,device=device)
print(accuracy)

avg val loss is 1.852087 and accuracy is 36.10%
36.1
